# __Data Cleaning for Netflix Shows__

### Importing:

In [1]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
from datetime import time
from sklearn.linear_model import LinearRegression, LogisticRegression
import re

In [2]:
pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# __Data Collection & Cleaning:__

In [3]:
#import the dataset with all ratings from IMBb. 
#note that each rating has a unique ID, not the show/movie title
ratings_df= pd.read_table("title.ratings.tsv")

#import the dataset with movie title given the ID
titles_df= pd.read_table("title.basics.tsv")

In [5]:
#perform an SQL join to obtain a dataframe with the rating, number of votes for that rating, and title
merged_ratings_df= duckdb.sql("""SELECT r.tconst, r.averageRating, r.numVotes, t.genres,
t.originalTitle AS Title
FROM ratings_df r, titles_df t
WHERE r.tconst=t.tconst""").df()

#drop the duplicate titles to prevent duplicate rows when merging with Netflix dataframe
merged_ratings_df = merged_ratings_df.drop_duplicates(subset=['Title'])

We observe that in merged_ratings_df, the movie titles are truly titles, which is what we expected. However, many rows do not have show titles, only episode numbers. For example, the observation with the ID of tt0740721 has the corresponding Title, Episode #1.1.  
We attempted to fix this problem with show titles by looking for IMDb dataset(s) that would let us link the ID and episode number with a show title. We found another IMDb source, title.akas.tsv, that was stated to have the ID and title of the show, but after reading it in, we realized it did not provide the true titles. We then found another IMDb source, title.episode.tsv, that provided the ID, the show's season and episode number, and another parent ID. However, we could not locate the data source allowing us to reference the parent ID with the title. At this point, we made the decision to proceed with films only. 

In [6]:
#import the netflix engagement reports
#import 1st half of 2023 netflix data
all_jan_jun_2023= pd.read_excel("What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.xlsx",
                                  sheet_name="Engagement",
                                  skiprows=5)

#import 2nd half of 2023 netflix data
movies_jul_dec_2023= pd.read_excel("What_We_Watched_A_Netflix_Engagement_Report_2023Jul-Dec.xlsx",
                                  sheet_name="Film",
                                  skiprows=5)

#import the 1st half of 2024 netflix data
movies_jan_jun_2024= pd.read_excel("What_We_Watched_A_Netflix_Engagement_Report_2024Jan-Jun.xlsx",
                                   sheet_name= "Film",
                                   skiprows=5)

#import the 2nd half of 2024 netflix data
movies_jul_dec_2024= pd.read_excel("What_We_Watched_A_Netflix_Engagement_Report_2024Jul-Dec.xlsx",
                                   sheet_name= "Film",
                                   skiprows=5)
movies_jul_dec_2024['Time_Period']= "2024 H2"
#import the 1st half of 2025 netflix data
movies_jan_jun_2025= pd.read_excel("What_We_Watched_A_Netflix_Engagement_Report_2025Jan-Jun.xlsx",
                                   sheet_name= "Movies",
                                   skiprows=5)

#source used to figure out how to skip rows when importing Excel sheet:
#https://www.statology.org/pandas-read-excel-skip-rows/

In [7]:
#add the time periods of the observations
all_jan_jun_2023['Time_Period']= "2023 H1"
movies_jul_dec_2023['Time_Period']= "2023 H2"
movies_jan_jun_2024['Time_Period']= "2024 H1"
movies_jan_jun_2025['Time_Period']= "2025 H1"

In [8]:
#combine all these dataframes into 1 Netflix dataframe, named netflix_df
#note that the Release Date has several missing values and thus omit it
netflix2023= duckdb.sql("""SELECT 
                    Title, 
                    "Available Globally?" AS Global, 
                    "Hours Viewed" AS Hours_Viewed, 
                    Runtime, Views, Time_Period
                FROM movies_jul_dec_2023 
                UNION 
                SELECT
                    Title, 
                    "Available Globally?" AS Global, 
                    "Hours Viewed" AS Hours_Viewed, 
                    NULL AS Runtime, 
                    NULL AS Views, Time_Period
                FROM all_jan_jun_2023""").df()

print(netflix2023.shape)

jan_jun_2024= duckdb.sql("""SELECT
                                Title, 
                                "Available Globally?" AS Global, 
                                "Hours Viewed" AS Hours_Viewed, 
                                Runtime, Views, Time_Period
                            FROM movies_jan_jun_2024""").df()

jul_dec_2024= duckdb.sql("""SELECT
                                Title, 
                                "Available Globally?" AS Global, 
                                "Hours Viewed" AS Hours_Viewed, 
                                Runtime, Views, Time_Period
                            FROM movies_jul_dec_2024""").df()

netflix2024= duckdb.sql("""SELECT * FROM jan_jun_2024 UNION ALL SELECT * FROM jul_dec_2024""").df()
#print(len(movies_jan_jun_2024) + len(movies_jul_dec_2024))
print(netflix2024.shape)

netflix2025= duckdb.sql("""SELECT
                                Title, 
                                "Available Globally?" AS Global, 
                                "Hours Viewed" AS Hours_Viewed, 
                                Runtime, Views, Time_Period
                            FROM movies_jan_jun_2025""").df()


print(netflix2025.shape)

netflix_df= duckdb.sql("""SELECT * FROM netflix2023 UNION SELECT * FROM netflix2024 UNION SELECT * 
FROM netflix2025""").df()

print(netflix_df.shape)

#UNION operator source: https://www.w3schools.com/sql/sql_union.asp

(27609, 6)
(18040, 6)
(8674, 6)
(54323, 6)


At this point, we have created a dataframe, netflix_df, that includes every film's viewership from the start of 2023 until the first half of 2025. We also printed the shapes of every dataframe to confirm that the netflix_df equals the sum of each year's number of observations.  
Next, we want to match each film with its IMDb rating, if a rating exists. Unfortunately, the Netflix and IMDb sources do not have a standardized convention for their film titles. In the following several code cells, we make an effort to clean the titles as much as possible, to improve the number of matches when we merge the Netflix and ratings dataframes.

In [9]:
#Netflix films with foreign language titles have both english translation then the foreign title
#cut the title of netflix shows with foreign language titles, keeping the english title only
mod_title= netflix_df['Title']
mod_title= mod_title.str.replace(r" \/\/.*", "", regex=True)

netflix_df['Title']= mod_title

#check that the foreign language component is gone
#netflix_df.head()

#One of us previously did similar work with meta characters and string detect/string replace in R,
#but used help with the string replace:
#https://stackoverflow.com/questions/5658369/how-to-input-a-regex-in-string-replace

In [10]:
#look at netflix titles
netflix_df.to_csv("netflix_titles.csv", index=False)
merged_ratings_df.to_csv("imdb_titles.csv", index=False)

In [11]:
#some Netflix titles end with the year in parenthesis, which is not seen in ratings titles

year_tester= netflix_df["Title"].str.endswith(")")

#removing the last 6 characters cuts the (Year) component if it is at the end of the title
netflix_df.loc[year_tester, "Title"]= netflix_df.loc[year_tester, "Title"].str[:-6]

#check that the (Year) component is gone
#year_tester_2= netflix_df["Title"].str.endswith(")")
#netflix_df[year_tester_2].head()

#source used for the str.endswith:
#https://www.w3schools.com/python/ref_string_endswith.asp

The netflix_df will now have more matches with the merged_ratings_df. We will now merge the two datasets with an INNER JOIN so that only the netflix films with ratings in IMDb are kept.

In [12]:
#Inner join Netflix and ratings datasets
netflix_ratings_df= duckdb.sql("""SELECT netflix_df.Title, netflix_df.Global, 
netflix_df.Hours_Viewed, netflix_df.Runtime, netflix_df.Views, netflix_df.Time_Period, 
merged_ratings_df.averageRating, merged_ratings_df.numVotes, merged_ratings_df.genres
FROM netflix_df
INNER JOIN merged_ratings_df
ON netflix_df.Title= merged_ratings_df.Title
""").df()
print(netflix_ratings_df.shape)

(26413, 9)


Next, we will clean the netflix_ratings_df to make it ready for analysis. Please see below.

In [13]:
#Grouping by titles to drop the duplicates.
#Taking the row with the maximum views because this is the row with peak popularity 
netflix_ratings_df = netflix_ratings_df.sort_values(by=["Views"], ascending=False)
netflix_ratings_df = netflix_ratings_df.drop_duplicates(["Title"], keep="first")
print(f"Our dataset has {netflix_ratings_df.shape[0]} number of films")

Our dataset has 8772 number of films


The 2023 data only has hours viewed, not viewership. We will fill the missing values with the hours viewed divided by the average runtime. We are choosing to do this so that we do not have to drop the 2023 data nor have empty data values.  
The Hours Viewed has the accurate data, and dividing it by the average runtime which is a constant means the transformation is not skewed. The average runtime is the best choice to transform the hours viewed because the runtime of a movie is unlikely to be extremely different in 2023-2025 compared to just 2023. 

In [14]:
#making time column to have type time, then adjusting runtime to be in minutes
netflix_ratings_df["Runtime"] = pd.to_datetime(netflix_ratings_df['Runtime'], format='%H:%M')
netflix_ratings_df["Runtime"] = netflix_ratings_df[
    "Runtime"].dt.hour*60 + netflix_ratings_df["Runtime"].dt.minute

#Finding average runtime and plugging it in to the missing values of runtime without skewing the data 
#to complete the dataframe enabling us to make future analysis
avgRuntime = np.round(np.mean(netflix_ratings_df["Runtime"]))
isEmptyRuntime = netflix_ratings_df["Runtime"].isna()
netflix_ratings_df.loc[isEmptyRuntime, "Runtime"] = avgRuntime

#Calculating views for empty values for Views column using runtime and hours viewed
isEmptyViews = netflix_ratings_df["Views"].isna()
netflix_ratings_df.loc[isEmptyViews, "Views"] = netflix_ratings_df["Hours_Viewed"]/np.round(
    avgRuntime/60)

To finish the dataset preparation, we will create a couple more columns to be used in future analysis.

In [15]:
#create new columns to use in future binary analysis on global avaiability 
netflix_ratings_df["Is_Global"] = netflix_ratings_df["Global"].map({'Yes': 1, 'No': 0})

#create new column that determines popularity based on being above and below average views
netflix_ratings_df["Popularity"] = netflix_ratings_df["Views"] > np.mean(netflix_ratings_df["Views"])
netflix_ratings_df["Popularity"] = netflix_ratings_df["Popularity"].map({True: int(1), False: int(0)})

#create new column that determines the number of reviews (numVotes) to be greater than 
#or less than the median
netflix_ratings_df["Sufficient Votes"] = netflix_ratings_df["numVotes"] > np.median(
    netflix_ratings_df["numVotes"])
netflix_ratings_df["Sufficient Votes"] = netflix_ratings_df["Sufficient Votes"].map(
    {True: int(1), False: int(0)})

Here, we are creating new columns for 10 different genres (Romance, Comedy, Animation, Documentary, Sport, Horror, Fantasy, Short, Drama, News).  If the specific movie has a genre, then there is a 1 as the boolean value; if not, then 0.  This will help with future genre analysis.

In [23]:
#creating the columns for 10 different genres, containing 1 for True and 0 for False
netflix_ratings_df["Genre: Romance"] = netflix_ratings_df["genres"].str.contains("Romance")
netflix_ratings_df["Genre: Romance"] = netflix_ratings_df["Genre: Romance"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Comedy"] = netflix_ratings_df["genres"].str.contains("Comedy")
netflix_ratings_df["Genre: Comedy"] = netflix_ratings_df["Genre: Comedy"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Animation"] = netflix_ratings_df["genres"].str.contains("Animation")
netflix_ratings_df["Genre: Animation"] = netflix_ratings_df["Genre: Animation"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Documentary"] = netflix_ratings_df["genres"].str.contains("Documentary")
netflix_ratings_df["Genre: Documentary"] = netflix_ratings_df["Genre: Documentary"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Sport"] = netflix_ratings_df["genres"].str.contains("Sport")
netflix_ratings_df["Genre: Sport"] = netflix_ratings_df["Genre: Sport"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Horror"] = netflix_ratings_df["genres"].str.contains("Horror")
netflix_ratings_df["Genre: Horror"] = netflix_ratings_df["Genre: Horror"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Fantasy"] = netflix_ratings_df["genres"].str.contains("Fantasy")
netflix_ratings_df["Genre: Fantasy"] = netflix_ratings_df["Genre: Fantasy"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Short"] = netflix_ratings_df["genres"].str.contains("Short")
netflix_ratings_df["Genre: Short"] = netflix_ratings_df["Genre: Short"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: Drama"] = netflix_ratings_df["genres"].str.contains("Drama")
netflix_ratings_df["Genre: Drama"] = netflix_ratings_df["Genre: Drama"].map({True: int(1), False: int(0)})

netflix_ratings_df["Genre: News"] = netflix_ratings_df["genres"].str.contains("News")
netflix_ratings_df["Genre: News"] = netflix_ratings_df["Genre: News"].map({True: int(1), False: int(0)})

In [24]:
#Creating a data-cleansed CSV file for our dataset
netflix_ratings_df.to_csv("netflix_ratings.csv", index=False)


#let's look at our cleaned dataset
print(netflix_ratings_df.head())

                Title Global  Hours_Viewed  Runtime      Views Time_Period  \
166    Back in Action    Yes     313000000    114.0  164700000     2025 H1   
4039           Damsel    Yes     263700000    110.0  143800000     2024 H1   
15719        Carry-On    Yes     274500000    120.0  137300000     2024 H2   
4140        The Union    Yes     238200000    109.0  131100000     2024 H2   
9129             Lift    Yes     230800000    107.0  129400000     2024 H1   

       averageRating  numVotes                 genres  Is_Global  ...  \
166              5.9     62144          Action,Comedy          1  ...   
4039             6.7        41           Comedy,Drama          1  ...   
15719            6.5    181168  Action,Crime,Thriller          1  ...   
4140             8.4        24           Comedy,Drama          1  ...   
9129             6.6        14                  Drama          1  ...   

       Genre: Romance  Genre: Comedy  Genre: Animation  Genre: Documentary  \
166           